# FDA Project

In [13]:
#Begin by importing pandas, regular expressions, and numpy
import pandas as pd
import re
import numpy as np
import json
import datetime
from bs4 import BeautifulSoup as bs
import requests
import re
import urllib
import time
import os


#If selenium is giving issue, use pip install selenium on its own line first
#pip install webdriver-manager

DOMAIN = "https://open.fda.gov/data/downloads/"
URL = "https://open.fda.gov/data/downloads/"

In [2]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [4]:
#The FDA website does not load the doanloadable files unless you scroll to that area of the page first
#Web-Scraping involves using the Selenium webdriver to open the site with Chrome, navigate to the
    #needed area, and hit the correct buttons at the correct time

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import selenium.common.exceptions
from selenium import webdriver
import time

from selenium.webdriver.support.wait import WebDriverWait

options = webdriver.ChromeOptions()

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.google.com")

driver.get(URL)
driver.maximize_window()

time.sleep(1)

button1 = driver.find_element(By.CLASS_NAME, "button.bg-primary.clr-white")
button1.click()

time.sleep(1)
 
element_link=WebDriverWait(driver, 10).until(EC.presence_of_element_located(
   (By.XPATH, '//*[@id="Medical Device Event"]')))

driver.execute_script("arguments[0].scrollIntoView(true)", element_link)

time.sleep(1)

button2 = driver.find_element(By.XPATH, '//*[@id="Medical Device Event"]/section/button')
button2.click()

time.sleep(1)

html = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
#print (html)

driver.close()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\Allen\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [5]:
#Snip HTML to just the portion in question
pattern = "Hide(.*?)All other data"
#pattern = "Hide(.*?)download.open.fda.gov/device/event/2012q1/device-event-0002-of-0002.json.zip"
substring = re.search(pattern, html).group(1)

In [6]:
#Itemize links into array
import lxml.html

url_list = lxml.html.fromstring(substring)
url_list = url_list.xpath('//a/@href')
#print(newlist)

In [7]:
start_year = 2000
end_year = 2021

year_list = list(range(start_year, end_year+1))

index_to_download = []

for meh in year_list:
    for bleh in range(0, len(url_list)):
        if str(meh) in url_list[bleh]:
            index_to_download.append(url_list.index(url_list[bleh]))

index_count = len(index_to_download)

In [8]:
#Follow links in array to download/process ZIPs

#Specify Download Path
path = 'C:/Users/Allen/Documents/FDA'
#pathinverse = 'C:\Users\Allen\Documents\FDA'
import requests, zipfile
from io import BytesIO

#Filter Data by Project Code, use "All" to include all data
pcode = "MDS"

#Run loop, opening JSONs
loopnumber = 0
datamain = ""
for snuh in index_to_download:
    print('Download ' + (str(loopnumber+1)) + " of " + (str(index_count)) + " started ")
    url = url_list[snuh]
    import requests, zipfile
    req = requests.get(url)
    print('Download ' + (str(loopnumber+1))+ " completed ")
    zipfile = zipfile.ZipFile(BytesIO(req.content))
    #filename = "FDA" + str(snuh+1)
    filename = "FDAdata.json"
    for i, f in enumerate(zipfile.filelist):
        f.filename = filename.format(i)
        zipfile.extract(f)
    print('File ' + (str(loopnumber+1))+ ' extracted')
    data = json.load(open(r'C:\Users\Allen\Documents\FDA\FDAdata.json'))
    data = data["results"]
    datamain = data
    if loopnumber == 0:
        print('Creating Dataframe with JSON ' + (str(loopnumber+1)))
        dfmain = pd.json_normalize(data,
                  record_path = "device",
                  meta = ["report_number","report_source_code","date_received","event_type","type_of_report","mdr_text"],
                  record_prefix = "_",
                  errors = "ignore")
        if pcode != "All":
            dfmain = dfmain[dfmain["_device_report_product_code"] == pcode]
        print('Dataframe Created')
    else:
        print('Appending Dataframe with JSON ' + (str(loopnumber+1)))
        dfnew = pd.json_normalize(data,
                  record_path = "device",
                  meta = ["report_number","report_source_code","date_received","event_type","type_of_report","mdr_text"],
                  record_prefix = "_",
                  errors = "ignore")
        if pcode != "All":
            dfnew = dfnew[dfnew["_device_report_product_code"] == pcode]
        dfmain = pd.concat([dfmain, dfnew])
        #dfmain.append(dfnew)
        print('JSON ' + (str(loopnumber+1)) + ' appended')
        
    os.remove(path + "/" + filename)
    loopnumber = loopnumber + 1
print("Dataframe ready")

#dfmain.to_csv(r'data.csv', index = False)
#print("Export file saved as 'data.csv'")

Download 1 of 176 started 
Download 1 completed 
File 1 extracted
Creating Dataframe with JSON 1
Dataframe Created
Download 2 of 176 started 
Download 2 completed 
File 2 extracted
Appending Dataframe with JSON 2
JSON 2 appended
Download 3 of 176 started 
Download 3 completed 
File 3 extracted
Appending Dataframe with JSON 3
JSON 3 appended
Download 4 of 176 started 
Download 4 completed 
File 4 extracted
Appending Dataframe with JSON 4
JSON 4 appended
Download 5 of 176 started 
Download 5 completed 
File 5 extracted
Appending Dataframe with JSON 5
JSON 5 appended
Download 6 of 176 started 
Download 6 completed 
File 6 extracted
Appending Dataframe with JSON 6
JSON 6 appended
Download 7 of 176 started 
Download 7 completed 
File 7 extracted
Appending Dataframe with JSON 7
JSON 7 appended
Download 8 of 176 started 
Download 8 completed 
File 8 extracted
Appending Dataframe with JSON 8
JSON 8 appended
Download 9 of 176 started 
Download 9 completed 
File 9 extracted
Appending Dataframe w

Download 70 completed 
File 70 extracted
Appending Dataframe with JSON 70
JSON 70 appended
Download 71 of 176 started 
Download 71 completed 
File 71 extracted
Appending Dataframe with JSON 71
JSON 71 appended
Download 72 of 176 started 
Download 72 completed 
File 72 extracted
Appending Dataframe with JSON 72
JSON 72 appended
Download 73 of 176 started 
Download 73 completed 
File 73 extracted
Appending Dataframe with JSON 73
JSON 73 appended
Download 74 of 176 started 
Download 74 completed 
File 74 extracted
Appending Dataframe with JSON 74
JSON 74 appended
Download 75 of 176 started 
Download 75 completed 
File 75 extracted
Appending Dataframe with JSON 75
JSON 75 appended
Download 76 of 176 started 
Download 76 completed 
File 76 extracted
Appending Dataframe with JSON 76
JSON 76 appended
Download 77 of 176 started 
Download 77 completed 
File 77 extracted
Appending Dataframe with JSON 77
JSON 77 appended
Download 78 of 176 started 
Download 78 completed 
File 78 extracted
Appendi

Appending Dataframe with JSON 137
JSON 137 appended
Download 138 of 176 started 
Download 138 completed 
File 138 extracted
Appending Dataframe with JSON 138
JSON 138 appended
Download 139 of 176 started 
Download 139 completed 
File 139 extracted
Appending Dataframe with JSON 139
JSON 139 appended
Download 140 of 176 started 
Download 140 completed 
File 140 extracted
Appending Dataframe with JSON 140
JSON 140 appended
Download 141 of 176 started 
Download 141 completed 
File 141 extracted
Appending Dataframe with JSON 141
JSON 141 appended
Download 142 of 176 started 
Download 142 completed 
File 142 extracted
Appending Dataframe with JSON 142
JSON 142 appended
Download 143 of 176 started 
Download 143 completed 
File 143 extracted
Appending Dataframe with JSON 143
JSON 143 appended
Download 144 of 176 started 
Download 144 completed 
File 144 extracted
Appending Dataframe with JSON 144
JSON 144 appended
Download 145 of 176 started 
Download 145 completed 
File 145 extracted
Appendin

In [9]:
#dfmain.info()

#Keep only relevant columns
dfmain = dfmain[["_device_report_product_code","_brand_name","_generic_name","_manufacturer_d_name","type_of_report","report_number","report_source_code",
                 "date_received","event_type","mdr_text"]]

#Rename columns
dfmain.columns = ["product_code","brand_name","generic_name","manufacturer_name","type_of_report","report_number",
                        "report_source_code","date_received","event_type","mdr_text"]

#Update date column to date format
dfmain["date_received"] = pd.to_datetime(dfmain["date_received"])

#Remove brackets from type of report column
dfmain['type_of_report'] = dfmain['type_of_report'].str.join(', ')

In [10]:
dfmain.shape[0]

249734

In [11]:
#pcode = "MDS"
#dftest = dfmain[dfmain["product_code"] == pcode]
#dftest.shape[0]
dfmain.to_csv(r'data.csv', index = False)

In [12]:
dfmain = pd.read_csv("data.csv", encoding = 'unicode_escape')